In [ ]:
#installation du package selenium
!pip install selenium
#installation du chromedriver
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (107.0.5304.87-0ubuntu11.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [ ]:
from selenium import webdriver
import pandas as pd

In [ ]:
#options chromedriver pour l'utiliser dans un notebook
options = webdriver.ChromeOptions()
options.add_argument("-headless")
options.add_argument("-no-sandbox")
options.add_argument("-disable-dev-shm-usage")
options.add_argument("enable-automation")
options.add_argument("--window-size = 1920, 1080")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument("--disable-gpu")

In [ ]:
#création de l'instance WebDriver de Chrome
driver = webdriver.Chrome("chromedriver", options = options) #options définie au-dessus
#lien du site
website = "https://larevueia.fr/"
#accès au site
driver.get(website)

In [ ]:
#liste des catégories du site La Revue IA
categories = ["ethique", "nlp", "evenements", "ml-dl", "data-science", "vision"]

In [ ]:
#liste des liens vers les articles
articles_links = []
#parcours des catégories
for category in categories:
  #lien de chaque catégorie
  category_link = website + category #fonctionne pour ce site en particulier
  driver.get(category_link) #pour ensuite pouvoir simuler un clic sur une catégorie et réitérer l'opération
  #récupération des liens href
  href_links = driver.find_elements("xpath", "//h2/a[@href]") #en html: les href sont les balises contenant les liens
  #ajout des leins avec le label de la catégorie
  for href_link in href_links:
    articles_links.append([href_link.get_attribute("href"), category])

In [ ]:
#affichage des 5 1ers liens d'article
articles_links[:5]

[['https://larevueia.fr/quest-ce-que-lintelligence-artificielle-decentralisee/',
  'ethique'],
 ['https://larevueia.fr/quest-ce-quun-reseau-lstm/', 'ethique'],
 ['https://larevueia.fr/lintelligence-artificielle-au-service-du-recrutement/',
  'ethique'],
 ['https://larevueia.fr/classification-de-tweets-en-direct-avec-apache-kafka-et-tweepy/',
  'ethique'],
 ['https://larevueia.fr/quels-liens-entre-blockchain-et-intelligence-artificielle/',
  'ethique']]

In [ ]:
df_articles = pd.DataFrame()
#parcours des liens collectés
for article_link in articles_links: #[lien, catégorie]
  #accès à la page de l'article
  driver.get(article_link[0]) #pour avoir le lien et pas la catégorie de l'article
  #récupération du contenue de l'article
  content_scrap = driver.find_elements("xpath", "//h1")
  if (len(content_scrap) != 0):
    content = content_scrap[0].text
  #création de la liste de nos données
  article = {"category" : article_link[1], "content" : content}
  #transformation en data frame
  df_article = pd.DataFrame(article, index = [0]) #pour avoir uniquement le contenu et la catégorie, sans index pour ne pas fausser
  #transformation en data frame
  df_articles = df_articles.append(df_article, ignore_index = True) #pour avoir uniquement le contenu et la catégorie, sans index pour ne pas fausser

In [ ]:
#afficher les 5 1ères lignes
df_articles.head()

,category,content
0,ethique,Qu’est-ce que l’intelligence artificielle déce...
1,ethique,Qu’est ce qu’un réseau LSTM ?
2,ethique,L’intelligence artificielle au service du recr...
3,ethique,Tutoriel Apache Kafka : classification de twee...
4,ethique,Quels liens entre blockchain et intelligence a...


In [ ]:
#afficher les 5 dernières lignes
df_articles.tail()

,category,content
55,vision,DeOldify : coloriser vos anciennes photos avec...
56,vision,DALL-E : génération d’images à partir de textes
57,vision,Comment le fisc français utilise l’intelligenc...
58,vision,L’intelligence artificielle pour la fécondatio...
59,vision,Qu’est-ce que la segmentation d’images ?


In [ ]:
#convertir le data frame en csv
df_articles.to_csv("la_revue_ia_articles.csv")